In [2]:
import os
import time
import requests

ELEFAN_PRODUCT_API_URL = os.environ.get("API_URL")
OBF_API_URL = "https://fr.openbeautyfacts.org/api/v2/product"

ELEFAN_HYGIENE_FAMILLE_CODE_TO_INCLUDE = [
    21,  # bebe (inclus nourriture...)
    22,  # hygiène, beauté
    25,  # santé
]

In [4]:
"""
Récupérer les produits depuis Elefan (via l'API)
"""
print("Elefan Grenoble")
elefan_product_list = list()

response = requests.get(ELEFAN_PRODUCT_API_URL)
elefan_product_list = response.json()
print("Nombre d'articles (dans KASO) :", len(elefan_product_list))
# print(elefan_product_list[0])

elefan_product_actifs_list = [p for p in elefan_product_list if p['status'] == 'ACTIF']
print("Nombre d'articles actifs (status = ACTIF dans KASO) :", len(elefan_product_actifs_list))

elefan_product_actifs_ean_list = [p for p in elefan_product_actifs_list if (len(str(p['code'])) in [8, 13])]
print("Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) :", len(elefan_product_actifs_ean_list))

elefan_product_actifs_ean_filtered_list = [p for p in elefan_product_actifs_ean_list if p['famille']['code'] in ELEFAN_HYGIENE_FAMILLE_CODE_TO_INCLUDE]
print("Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) :", len(elefan_product_actifs_ean_filtered_list))

Elefan Grenoble
Nombre d'articles (dans KASO) : 2106
Nombre d'articles actifs (status = ACTIF dans KASO) : 1456
Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) : 1124
Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) : 137


In [5]:
print(f"Lien avec OBF (pour les {len(elefan_product_actifs_ean_filtered_list)} articles filtrés)")

found = 0
for index, elefan__product in enumerate(elefan_product_actifs_ean_filtered_list):
    response = requests.get(f"{OBF_API_URL}/{elefan_product['code']}.json")
    if response.status_code == 200:
        found += 1
        elefan_product_actifs_ean_filtered_list[index]['OFF'] = response.json()
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 50) == 0:
        print(f"progress: {index}...")

print("Trouvés", found, "/", len(elefan_product_actifs_ean_filtered_list))

Lien avec OBF (pour les 137 articles filtrés)


NameError: name 'elefan_product' is not defined